In [1]:
# pip install arff

In [0]:
import arff
import numpy as np
import seaborn as sn
import matplotlib.pyplot as plt
from operator import attrgetter, itemgetter
from io import StringIO
from sklearn.model_selection import train_test_split
import nltk
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn import tree
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import make_scorer, confusion_matrix,classification_report,precision_recall_fscore_support as score, average_precision_score
from sklearn import metrics
from sklearn.model_selection import cross_validate
from pickle import Pickler, Unpickler

## training model
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.ensemble import RandomForestClassifier, VotingClassifier
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import SGDClassifier
from sklearn.tree import DecisionTreeClassifier

import matplotlib.pyplot as plt
from sklearn.model_selection import GridSearchCV

In [3]:
from google.colab import drive
drive.mount("/content/gdrive", force_remount=True)

Mounted at /content/gdrive


In [4]:
root_path = "gdrive/My Drive/Colab Notebooks/"
df = Unpickler(open(root_path + "dataset/OffComBR3.sav", 'rb')).load()

X = df['sentence'].tolist()
y = df['hate'].tolist()


nltk.download('stopwords')
nltk.download('rslp')
stopwords = nltk.corpus.stopwords.words('portuguese')
stemmer = nltk.stem.RSLPStemmer()
nltk.download('punkt')

def clean_text(txt):
    text = ''
    for w in nltk.word_tokenize(txt):
        if w not in stopwords:
            text = text + stemmer.stem(w) + ' '
    return text.strip()


for i in range(len(X)):
    n_txt = clean_text(X[i])
    X[i] = n_txt

X_train, X_test, y_train, y_test = train_test_split(
                                X, y, test_size=0.33, random_state=42)

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Package rslp is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [0]:


parameters_rf = {'tfidf__ngram_range': [(1,1)],
                 'tfidf__max_df': [0.2],
                 'tfidf__min_df': [2],
                 'tfidf__use_idf': [True],
                 'tfidf__smooth_idf': [False],
                 'tfidf__sublinear_tf': [True],
                 'tfidf__strip_accents': ['ascii'],
                 'clf__n_estimators': [10, 100, 1000],
                 'clf__max_depth': [None],
                 'clf__min_samples_split': [2, 4],
                 'clf__min_samples_leaf': [1, 2, 4],
                 'clf__class_weight': [{0:1, 1:3}]
                }

parameters = [parameters_rf]

In [0]:
classifiers = [RandomForestClassifier()]

In [15]:
for i, classifier in enumerate(classifiers):
  cl = Pipeline([
          ('tfidf',TfidfVectorizer()),
          ('clf', classifier),
        ])

  gs_clf = GridSearchCV(cl, parameters[i], cv=10, iid=False, n_jobs=-1, scoring='recall')

  gs_clf.fit(X, y)
  print(classifier.__class__.__name__)
  print(gs_clf.best_score_)
  print(gs_clf.best_params_)

RandomForestClassifier
0.4521428571428571
{'clf__class_weight': {0: 1, 1: 3}, 'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 4, 'clf__n_estimators': 10, 'tfidf__max_df': 0.2, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1), 'tfidf__smooth_idf': False, 'tfidf__strip_accents': 'ascii', 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True}


In [16]:
for i, classifier in enumerate(classifiers):
  cl = Pipeline([
          ('tfidf',TfidfVectorizer()),
          ('clf', classifier),
        ])

  gs_clf = GridSearchCV(cl, parameters[i], cv=10, iid=False, n_jobs=-1, scoring='precision')

  gs_clf.fit(X, y)
  print(classifier.__class__.__name__)
  print(gs_clf.best_score_)
  print(gs_clf.best_params_)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RandomForestClassifier
0.5611011904761904
{'clf__class_weight': {0: 1, 1: 3}, 'clf__max_depth': None, 'clf__min_samples_leaf': 1, 'clf__min_samples_split': 2, 'clf__n_estimators': 1000, 'tfidf__max_df': 0.2, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1), 'tfidf__smooth_idf': False, 'tfidf__strip_accents': 'ascii', 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True}


In [17]:
for i, classifier in enumerate(classifiers):
  cl = Pipeline([
          ('tfidf',TfidfVectorizer()),
          ('clf', classifier),
        ])

  gs_clf = GridSearchCV(cl, parameters[i], cv=10, iid=False, n_jobs=-1, scoring='accuracy')

  gs_clf.fit(X, y)
  print(classifier.__class__.__name__)
  print(gs_clf.best_score_)
  print(gs_clf.best_params_)

/usr/local/lib/python3.6/dist-packages/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


RandomForestClassifier
0.8043659091717343
{'clf__class_weight': {0: 1, 1: 3}, 'clf__max_depth': None, 'clf__min_samples_leaf': 2, 'clf__min_samples_split': 2, 'clf__n_estimators': 1000, 'tfidf__max_df': 0.2, 'tfidf__min_df': 2, 'tfidf__ngram_range': (1, 1), 'tfidf__smooth_idf': False, 'tfidf__strip_accents': 'ascii', 'tfidf__sublinear_tf': True, 'tfidf__use_idf': True}


In [0]:
## classificadores com os melhores hiperparametros

## recall
rf1 = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 2,
                                 ngram_range= (1, 1),
                                 smooth_idf= False,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', RandomForestClassifier(class_weight= {0:1, 1:3}, max_depth=None, min_samples_leaf= 1, min_samples_split= 4, n_estimators=10)),
        ])

rf2 = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 2,
                                 ngram_range= (1, 1),
                                 smooth_idf= False,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', RandomForestClassifier(max_depth=None, min_samples_leaf= 1, min_samples_split= 4, n_estimators=10)),
        ])

rf3 = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 2,
                                 ngram_range= (1, 1),
                                 smooth_idf= False,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', RandomForestClassifier(class_weight= {0:3, 1:1}, max_depth=None, min_samples_leaf= 1, min_samples_split= 4, n_estimators=10)),
        ])

##precision
rf4 = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 2,
                                 ngram_range= (1, 1),
                                 smooth_idf= False,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', RandomForestClassifier(class_weight= {0:1, 1:3}, max_depth=None, min_samples_leaf= 1, min_samples_split= 2, n_estimators=1000)),
        ])
rf5 = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 2,
                                 ngram_range= (1, 1),
                                 smooth_idf= False,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', RandomForestClassifier(max_depth=None, min_samples_leaf= 1, min_samples_split= 2, n_estimators=1000)),
        ])


## accuracy
rf6 = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 2,
                                 ngram_range= (1, 1),
                                 smooth_idf= False,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', RandomForestClassifier(class_weight= {0:1, 1:3}, max_depth=None, min_samples_leaf= 2, min_samples_split= 2, n_estimators=1000)),
        ])

rf7 = Pipeline([
        ('tfidf',TfidfVectorizer(max_df= 0.2,
                                 min_df= 2,
                                 ngram_range= (1, 1),
                                 smooth_idf= False,
                                 strip_accents= 'ascii',
                                 sublinear_tf= True,
                                 use_idf = True)),
        ('clf', RandomForestClassifier(max_depth=None, min_samples_leaf= 2, min_samples_split= 2, n_estimators=1000)),
        ])

improved_classifiers = [rf1, rf2, rf3, rf4, rf5, rf6, rf7]


In [22]:
for classifier in improved_classifiers:
    classifier.fit(X_train, y_train)

    print(classifier['clf'].__class__.__name__)
    ## Report normal
    pred = classifier.predict(X_test)
    print(classification_report(y_test, pred))

    ## Métricas do CV:


    recall = cross_val_score(classifier, X, y, cv=10, scoring='recall')
    print("Recall: %0.2f (+/- %0.2f) [%s]" % (recall.mean(), recall.std(), classifier['clf'].__class__.__name__))
    precision = cross_val_score(classifier, X, y, cv=10, scoring='precision')
    print("Precision: %0.2f (+/- %0.2f) [%s]" % (precision.mean(), precision.std(), classifier['clf'].__class__.__name__))
    accuracy = cross_val_score(classifier, X, y, cv=10, scoring='accuracy')
    print("Accuracy: %0.2f (+/- %0.2f) [%s]" % (accuracy.mean(), accuracy.std(), classifier['clf'].__class__.__name__))
    print("=================================================================")
  

RandomForestClassifier
              precision    recall  f1-score   support

           0       0.83      0.91      0.86       267
           1       0.48      0.31      0.38        74

    accuracy                           0.78       341
   macro avg       0.65      0.61      0.62       341
weighted avg       0.75      0.78      0.76       341

Recall: 0.41 (+/- 0.13) [RandomForestClassifier]
Precision: 0.46 (+/- 0.14) [RandomForestClassifier]
Accuracy: 0.76 (+/- 0.09) [RandomForestClassifier]
RandomForestClassifier
              precision    recall  f1-score   support

           0       0.83      0.96      0.89       267
           1       0.62      0.27      0.38        74

    accuracy                           0.81       341
   macro avg       0.73      0.61      0.63       341
weighted avg       0.78      0.81      0.78       341

Recall: 0.29 (+/- 0.10) [RandomForestClassifier]
Precision: 0.55 (+/- 0.23) [RandomForestClassifier]
Accuracy: 0.80 (+/- 0.04) [RandomForestClassifi